In [1]:
#@title Install and copy sample { form-width: "10%" }

# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
!pip -q install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install shapely
!unzip -q "/content/drive/My Drive/final_projects/icdar.zip"
!unzip -q '/content/drive/My Drive/final_projects/bankdata1.zip'
!ln -s '/content/drive/My Drive/work_dirs/' /content/workdir
!ln -s '/content/drive/My Drive/detectron2/' /content/detectron2
!ln -s '/content/drive/My Drive/final_projects/detection/' /content/detection

     |████████████████████████████████| 6.6MB 555kB/s 
     |████████████████████████████████| 2.2MB 4.5MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [8]:
#@title IMport { form-width: "10%" }

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
import matplotlib.pyplot  as plt

In [ ]:
#@title Verify Single { form-width: "10%" }
from detectron2.structures import BitMasks, Boxes, BoxMode, Keypoints, PolygonMasks, RotatedBoxes
import glob
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon)
cfg.MODEL.WEIGHTS = '/content/detection/savedmodel/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8   # set the testing threshold for this model
files = glob.glob('/content/detectron2/TestImages/*.jpg')
for fno,f in enumerate(files):
  im = cv2.imread(f)
  predictor = DefaultPredictor(cfg)
  print('Predicting',f)
  outputs = predictor(im)
  print('Predicting Done')
  MetadataCatalog.get("table_bank").set(thing_classes=["table"])
  table_metadata = MetadataCatalog.get("table_bank")
  # v = Visualizer(im[:, :, ::-1],
  #                 metadata=table_metadata, 
  #                 scale=0.8
  # )
  # out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  # cv2_imshow(out.get_image()[:, :, ::-1])

  predictions = outputs["instances"].to("cpu")
  boxesTn = predictions.pred_boxes.tensor.numpy() if predictions.has("pred_boxes") else None
  # scores = predictions.scores if predictions.has("scores") else None
  # classes = predictions.pred_classes.numpy() if predictions.has("pred_classes") else None
  if isinstance(boxesTn, Boxes) or isinstance(boxesTn, RotatedBoxes):
    boxes = boxesTn.tensor.numpy()
  else:
    boxes=np.asarray(boxesTn)
  for boxno in range(len(boxes)):
    x1,y1,x2,y3 = int(boxes[boxno][0]),int(boxes[boxno][1]),int(boxes[boxno][2]),int(boxes[boxno][3])
    crop = im[y1:y3,x1:x2]
    cv2.imwrite('/content/box_'+str(fno)+'_'+str(boxno)+'.jpg',crop)
    plt.imshow(crop)

In [14]:
#@title CRAFT IMPORT LOAD  MODEL{ form-width: "10%" }
import sys
import os
import time
import argparse

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

from PIL import Image

import cv2
from skimage import io
import numpy as np
import craft_utils
import imgproc
import file_utils
import json
import zipfile
from craft import CRAFT
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict
import h5py
import croputil 
import importlib
importlib.reload(croputil)
from croputil import *


train_model='/content/workdir/craft_ic15_20k.pth'

class args :
  trained_model = train_model
  cuda=True
  text_threshold=0.7
  low_text=0.4
  link_threshold=0.4
  canvas_size=1280
  mag_ratio=1.5
  poly=False
  show_time=False
  test_folder='/data/'
  refine=False
  refiner_model='weights/craft_refiner_CTW1500.pth'


def copyStateDict(state_dict):
    if list(state_dict.keys())[0].startswith("module"):
        start_idx = 1
    else:
        start_idx = 0
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = ".".join(k.split(".")[start_idx:])
        new_state_dict[name] = v
    return new_state_dict

def str2bool(v):
    return v.lower() in ("yes", "y", "true", "t", "1")

def test_net(net, image, text_threshold, link_threshold, low_text, cuda, poly, refine_net=None):
    t0 = time.time()

    # resize
    img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, args.canvas_size, interpolation=cv2.INTER_LINEAR, mag_ratio=args.mag_ratio)
    ratio_h = ratio_w = 1 / target_ratio

    # preprocessing
    x = imgproc.normalizeMeanVariance(img_resized)
    x = torch.from_numpy(x).permute(2, 0, 1)    # [h, w, c] to [c, h, w]
    x = Variable(x.unsqueeze(0))                # [c, h, w] to [b, c, h, w]
    if cuda:
        x = x.cuda()

    # forward pass
    with torch.no_grad():
        y, feature = net(x)

    # make score and link map
    score_text = y[0,:,:,0].cpu().data.numpy()
    score_link = y[0,:,:,1].cpu().data.numpy()

    # refine link
    if refine_net is not None:
        with torch.no_grad():
            y_refiner = refine_net(y, feature)
        score_link = y_refiner[0,:,:,0].cpu().data.numpy()

    t0 = time.time() - t0
    t1 = time.time()

    # Post-processing
    boxes, polys = craft_utils.getDetBoxes(score_text, score_link, text_threshold, link_threshold, low_text, poly)

    # coordinate adjustment
    boxes = craft_utils.adjustResultCoordinates(boxes, ratio_w, ratio_h)
    polys = craft_utils.adjustResultCoordinates(polys, ratio_w, ratio_h)
    for k in range(len(polys)):
        if polys[k] is None: polys[k] = boxes[k]

    t1 = time.time() - t1

    # render results (optional)
    render_img = score_text.copy()
    render_img = np.hstack((render_img, score_link))
    ret_score_text = imgproc.cvt2HeatmapImg(render_img)

    if args.show_time : print("\ninfer/postproc time : {:.3f}/{:.3f}".format(t0, t1))

    return boxes, polys, ret_score_text

def loadCraftModel():
  net = CRAFT()
  print('Loading weights from checkpoint (' + args.trained_model + ')')
  if args.cuda:
      net.load_state_dict(copyStateDict(torch.load(args.trained_model)))
  else:
      net.load_state_dict(copyStateDict(torch.load(args.trained_model, map_location='cpu')))
  if args.cuda:
      net = net.cuda()
      net = torch.nn.DataParallel(net)
      cudnn.benchmark = False
  net.eval()
  return net

net=loadCraftModel()


ModuleNotFoundError: ignored

In [11]:
%cd /content
!git clone https://github.com/clovaai/CRAFT-pytorch.git

/content
Cloning into 'CRAFT-pytorch'...
remote: Enumerating objects: 56, done.
remote: Total 56 (delta 0), reused 0 (delta 0), pack-reused 56
Unpacking objects: 100% (56/56), done.


In [12]:
%cd /content/CRAFT-pytorch/


/content/CRAFT-pytorch


ERROR: Could not find a version that satisfies the requirement torch==0.4.1.post2 (from -r requirements.txt (line 1)) (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2, 0.3.1, 0.4.0, 0.4.1, 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0)
ERROR: No matching distribution found for torch==0.4.1.post2 (from -r requirements.txt (line 1))
